In [1]:
import pandas as pd
from datetime import datetime, timezone

## Wazers - datetime,duplicates,groupBy

In [2]:
wazerCount = pd.read_csv('poctyUzivatelu.csv', encoding = 'utf-8', sep = ';', names = ['unixTime','wazers','alerts'])

In [3]:
wazerCount.head()

,unixTime,wazers,alerts
0,1594755900,397,0
1,1594756200,397,0
2,1594757700,299,0
3,1594758000,300,0
4,1594794900,2031,0


In [4]:
datetime.fromtimestamp(int(wazerCount.loc[0,['unixTime']])).strftime('%Y-%m-%d %H:%M:%S')# funkce na prevod unix

'2020-07-14 21:45:00'

In [5]:
dateTimeStrList = []
for i in wazerCount.index:
    datum = datetime.fromtimestamp(int(wazerCount.loc[i,['unixTime']])).strftime('%Y-%m-%d %H:%M:%S') #převod na lokální čas,pokud nezadám tz = timezone.utc do argum. fromtmpst
    dateTimeStrList.append(datum)

In [6]:
dateTimeList = []
for i in range(0,len(dateTimeStrList)):
    dateTimeList.append(datetime.strptime(dateTimeStrList[i], '%Y-%m-%d %H:%M:%S'))

In [ ]:
dateTimeList[:5]

In [7]:
weekDayList = []
weekDays = ("Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday")
for i in range (0,len(dateTimeList)):
    weekday = weekDays[dateTimeList[i].weekday()]
    weekDayList.append(weekday)

In [8]:
hourList = []
for i in range(0,len(dateTimeList)):
    hourList.append(dateTimeList[i].hour)

In [9]:
minutesList = []
for i in range(0,len(dateTimeList)):
    minutesList.append(dateTimeList[i].minute)

In [10]:
quarters = []
for i in range(0,len(dateTimeList)):
    if minutesList[i] <= 15:
        quarters.append('00')
    elif minutesList[i] <= 30:
        quarters.append('15')
    elif minutesList[i] <= 45:
        quarters.append('30')
    else:
        quarters.append('45')
quarters[:10]

['30', '45', '00', '15', '30', '30', '30', '45', '45', '00']

In [11]:
wazerCount['datum_cas'] = dateTimeList
wazerCount['weekday'] = weekDayList
wazerCount['hour'] = hourList
wazerCount['quarter'] = quarters
wazerCount['minutes'] = minutesList

In [12]:
wazerCount.head()
len(wazerCount)

25221

In [13]:
wazerCountFinal = wazerCount.sort_values('unixTime', ignore_index = True)

In [14]:
len(wazerCountFinal)
wazerCountFinal.head(20)

,unixTime,wazers,alerts,datum_cas,weekday,hour,quarter,minutes
0,1594591800,153,0,2020-07-13 00:10:00,Monday,0,00,10
1,1594592400,113,0,2020-07-13 00:20:00,Monday,0,15,20
2,1594593000,95,0,2020-07-13 00:30:00,Monday,0,15,30
3,1594593600,85,0,2020-07-13 00:40:00,Monday,0,30,40
4,1594594200,72,0,2020-07-13 00:50:00,Monday,0,45,50
5,1594594800,63,0,2020-07-13 01:00:00,Monday,1,00,0
6,1594595400,66,0,2020-07-13 01:10:00,Monday,1,00,10
7,1594596000,67,0,2020-07-13 01:20:00,Monday,1,15,20
8,1594596600,65,0,2020-07-13 01:30:00,Monday,1,15,30
9,1594597200,64,0,2020-07-13 01:40:00,Monday,1,30,40


In [15]:
wazerCountFinal = wazerCountFinal.drop_duplicates(subset = ['unixTime'], ignore_index = True)
len(wazerCountFinal)

25091

In [16]:
datum = []
for i in wazerCountFinal.index:
    date = wazerCountFinal['datum_cas'][i].date()
    datum.append(date)

In [17]:
wazerCountFinal['datum']= datum

group by hours/days

In [23]:
maxhWazers = wazerCountFinal.groupby(['datum','hour'])['wazers'].max()
maxdWazers = wazerCountFinal.groupby(['datum'])['wazers'].max()

In [24]:
maxhWazers = maxhWazers.rename('maxhWazers')
maxdWazers = maxdWazers.rename('maxdWazers')
maxdWazers

datum
2020-07-13    2776
2020-07-14    1342
2020-07-15    2733
2020-07-18    2518
2020-07-19     786
              ... 
2020-10-16    1994
2020-10-17     908
2020-10-18    1097
2020-10-19    1736
2020-10-20    1728
Name: maxdWazers, Length: 92, dtype: int64

In [25]:
wazerCountFinal= wazerCountFinal.merge(maxhWazers,how = 'left',on = ['datum','hour'])

In [26]:
wazerCountFinal = wazerCountFinal.merge(maxdWazers, how = 'left', on = ['datum'])

In [27]:
len(wazerCountFinal)

25091

In [28]:
wazerCountFinal

,unixTime,wazers,alerts,datum_cas,weekday,hour,quarter,minutes,datum,maxhWazers,maxdWazers
0,1594591800,153,0,2020-07-13 00:10:00,Monday,0,00,10,2020-07-13,153,2776
1,1594592400,113,0,2020-07-13 00:20:00,Monday,0,15,20,2020-07-13,153,2776
2,1594593000,95,0,2020-07-13 00:30:00,Monday,0,15,30,2020-07-13,153,2776
3,1594593600,85,0,2020-07-13 00:40:00,Monday,0,30,40,2020-07-13,153,2776
4,1594594200,72,0,2020-07-13 00:50:00,Monday,0,45,50,2020-07-13,153,2776
...,...,...,...,...,...,...,...,...,...,...,...
25086,1603203600,1717,220,2020-10-20 16:20:00,Tuesday,16,15,20,2020-10-20,1728,1728
25087,1603203900,1699,218,2020-10-20 16:25:00,Tuesday,16,15,25,2020-10-20,1728,1728
25088,1603204200,1728,228,2020-10-20 16:30:00,Tuesday,16,15,30,2020-10-20,1728,1728
25089,1603204500,1712,217,2020-10-20 16:35:00,Tuesday,16,30,35,2020-10-20,1728,1728


In [29]:
wazersMaxd = []

In [30]:
for i in wazerCountFinal.index:
    if (wazerCountFinal['hour'][i]== 1) & (wazerCountFinal['minutes'][i]== 10):
        wazersMaxd.append(wazerCountFinal['maxdWazers'][i])
    else:
        wazersMaxd.append('0')

In [31]:
wazersMaxd

['0',
 '0',
 '0',
 '0',
 '0',
 '0',
 2776,
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 1342,
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '

In [32]:
wazersMaxh = []
for i in wazerCountFinal.index:
    if wazerCountFinal['wazers'][i]==wazerCountFinal['maxhWazers'][i]:
        wazersMaxh.append(wazerCountFinal['maxhWazers'][i])
    else:
        wazersMaxh.append('0')

In [33]:
wazerCountFinal['maxhWazers']=wazersMaxh

In [34]:
wazerCountFinal['dWazers']=wazersMaxd

In [ ]:
wazerCountFinal.to_csv('wazerCountFinal3.csv', encoding ='utf-8', index = False)